In [62]:
import tushare as ts
import pandas as pd
import time
import datetime

pd.set_option('display.max_rows',50)
pd.set_option('display.max_columns',80)
pd.set_option('display.width',600)

t_str = '2017-08-11 13:00:00'
blackdate = datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
print(blackdate)

2017-08-11 13:00:00


In [63]:
simdf = pd.read_csv("./sim.txt", sep=' ')

In [65]:
codes = list(simdf['code'])
print(len(codes))

28


In [58]:
df = ts.get_realtime_quotes(codes)
data_list = []
for index,row in df.iterrows() :
    code = row['code'] if row['code'] not in ['000001'] else 'sh'
    pre_close = float(row['pre_close'])
    price = float(row['price'])
    price_diff = price - pre_close
    change_percent = price_diff / pre_close * 100
    
    index = list(simdf['code']).index(code)
    cost = simdf.ix[index,'cost']
    share = simdf.ix[index,'share']
    cost_diff = price - cost
    profit = (cost_diff) * share
    profit_percent = cost_diff / cost * 100.0
    datastr = str("%.3f%%"%change_percent) + ',' + str("%.3f"%cost_diff) + ',' + str("%.3f"%profit) + ',' + str("%.3f"%profit_percent)
    data_list.append([astr for astr in datastr.split(',')])
    df_append = pd.DataFrame(data_list, columns=['change_percent','cost_diff','profit','profit_percent'])
    

In [59]:
df = df.join(df_append)
df['profit_percent'] = df['profit_percent'].astype('float32')
df = df.sort_values('profit_percent', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

print(df[['code','name','price','change_percent','bid','ask','pre_close','open','low','high','time','cost_diff','profit','profit_percent']])


      code   name      price  change_percent     bid     ask  pre_close       open        low       high      time cost_diff     profit profit_percent
9   000935   四川双马      0.000        -100.000   0.000   0.000     23.060      0.000      0.000      0.000  14:53:00   -23.060  -2306.000      -100.000%
5   000672   上峰水泥     10.340          -9.139  10.340  10.350     11.380     11.200     10.300     11.230  14:53:51    -0.510    -51.000        -4.700%
3   000630   铜陵有色      2.990          -7.430   2.980   2.990      3.230      3.170      2.970      3.180  14:53:51    -0.060     -6.000        -1.967%
11  002114   罗平锌电     21.080          -7.014  21.070  21.080     22.670     21.960     20.810     22.180  14:53:51    -0.160    -16.000        -0.753%
15  600069   银鸽投资      8.720          -5.320   8.720   8.730      9.210      9.200      8.720      9.200  14:53:51    -0.170    -17.000        -1.912%
23  600908   无锡银行     11.120          -4.467  11.130  11.150     11.640     11.500     11.080 